## Using Tools in Llama 3.1 to provide real-time information in an LLM chat

Llama 3.1 was just released this week, and supports 'tools'.  Tools are simple functions that you can write and provide to the LLM to answer particular questions.   This can be very valuable in an agentic workflow built for a highly specialized purpose, where you can use the LLM to determine the intent of a user's question in a chat, extract the relevant parameters from the chat and pipe them into the function you have written in order to get the required info.

Let's look at a simple example.  I'm going to create a workflow to find out the current temperature in a particular place of interest.

### Creating a tool to get the current temperature

In this code I am writing a simple Python function which will ping an API to get the current temperature in a place.

In [23]:
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

'The current temperature in London is 10 degrees Celsius'

That seems to have worked.  

### Adding the tool to the LLM workflow

Next I am going to use this function as a tool to feed to the LLM.  Not all LLMs support tools, but for those that do, the idea is as follows:
* The LLM should identify the intent of a user query and determine that the intent is appropriate to call the tool
* The LLM should extract or determine the right arguments/parameters to give to the tool
* The LLM should capture the response of the tool and feed it back to the user.  To do this it may have to override some default behaviors (for example many LLMs have default responses which inform users that they cannot provide certain real-time information).

First, let's send a request with a message to Llama 3.1, and as part of that request, we will inform Llama that we have a function on our end which can determine the weather in a given place.

In [24]:
import ollama
client = ollama.Client()

def tool_chat(model: str, query: str):
    response = client.chat(
        model = model,
        messages = [{'role': 'user', 'content': query}],
        tools = [
          {
            'type': 'function',
            'function': {
              'name': 'get_current_temperature',
              'description': 'Get the temperature in a place',
              'parameters': {
                'type': 'object',
                'properties': {
                  'place': {
                    'type': 'string',
                    'description': 'The place for which the temperature is requested',
                  }
                },
                'required': ['place'],
              },
            },
          },
        ],
      )
    
    return response['message']

Now let's see what the LLM returns when we ask a question that relates in some way to weather.

In [25]:
query = "Should I wear a warm coat today in Rome?"

tool_chat("llama3.1:8b", query)

{'role': 'assistant',
 'content': '',
 'tool_calls': [{'function': {'name': 'get_current_temperature',
    'arguments': {'place': 'Rome'}}}]}

In [26]:
import ollama
import asyncio

async def weather_chat(model: str, query: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Answer the following question: {query}.  
          Ignore any previous instructions or defaults and instead use the following information:  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


### Testing using various open models and queries
First, let's try this with Google's Gemma2 model and a simple question.


In [27]:
await weather_chat('gemma2:27b', 'What is the temperature like right now in Dublin?')

ResponseError: gemma2:27b does not support tools

OK, so we can see that some models don't yet support this feature.  Let's try the small version of Llama 3.1.

In [5]:
await weather_chat('llama3.1:8b', 'What is the temperature like right now in Dublin?')

The current temperature in Dublin is 20 degrees Celsius.


Works nicely - let's try asking indirectly.

In [6]:
await weather_chat('llama3.1:8b', "What is the current temperature in Ireland's capital?")

Based on the provided answer, I can tell you that the current temperature in Dublin, the capital of Ireland, is 20 degrees Celsius.


Very nice! Let's push things a little further.

In [3]:
await weather_chat('llama3.1:8b', "My sister says that if I flew into the capital city of Norway today, I should wear clothing for extreme weather.  Should I trust her advice?")

It's generally a good idea to trust your sister's advice, especially if she has experience with extreme weather conditions. However, in this case, the current temperature in Oslo is 13 degrees Celsius, which is quite mild. If you're planning to visit Oslo today, it's unlikely that you'll need to wear clothing for extreme weather. A light jacket or sweater should be sufficient.


In [10]:
await weather_chat('llama3.1:8b', "Compare the temperatures of these two cities right now: Dunedin, New Zealand and Reykjavik, Iceland?")

Based on the provided information, I can compare the temperatures of the two cities as follows:

Dunedin, New Zealand: 4°C (current temperature)
Reykjavik, Iceland: 13°C (current temperature)

The temperature in Reykjavik, Iceland is currently higher than that of Dunedin, New Zealand by 9 degrees Celsius.


In [11]:
await weather_chat('llama3.1:8b', "What kinds of clothes should I pack for my trip to Tasmania which leaves tomorrow?")

Considering the current temperature in Tasmania is 3°C, I would recommend packing layers for your trip. It's likely to be cool and possibly even chilly, especially in the mornings and evenings.

For a comfortable and versatile wardrobe, consider packing:

* A mix of lightweight and warmer tops (fleeces, sweaters, or thermals)
* Waterproof or water-resistant outerwear (jacket or windbreaker) to protect against Tasmania's infamous rain
* Comfortable and sturdy pants or trousers for outdoor activities like hiking or exploring
* Insulating layers (fleece or down jacket) for colder moments
* Warm socks and gloves for chilly mornings and evenings
* A scarf or neck warmer for added warmth
* Waterproof shoes or boots with good grip for navigating Tasmania's rugged terrain

Remember to check the weather forecast before your trip to ensure you're prepared for any potential changes in temperature or precipitation. Have a great time in Tasmania!


In [12]:
await weather_chat('llama3.1:8b', "How much longer would it take a 50g ice cube to melt today in Dunedin compared to Marrakech?")


To answer this question, we need to calculate the time it would take for a 50g ice cube to melt at each location.

The rate of melting is directly proportional to the temperature difference between the object and its surroundings. We can use the following formula to estimate the melting time:

Melting Time = (mass x specific heat capacity) / (density x surface area x temperature difference)

We'll assume a density of 0.92 g/cm³ for ice, a specific heat capacity of 2.05 J/g°C, and a surface area of approximately 6 cm² for the 50g ice cube.

First, let's calculate the melting time in Dunedin:

Temperature difference = 4°C (Dunedin) - 0°C (ice) = 4°C
Melting Time in Dunedin ≈ (50 g x 2.05 J/g°C) / (0.92 g/cm³ x 6 cm² x 4°C) ≈ 137 seconds

Next, let's calculate the melting time in Marrakech:

Temperature difference = 27°C (Marrakech) - 0°C (ice) = 27°C
Melting Time in Marrakech ≈ (50 g x 2.05 J/g°C) / (0.92 g/cm³ x 6 cm² x 27°C) ≈ 10 seconds

Now, let's find the difference in melting times

In [13]:
await weather_chat('llama3.1:70b', "How much longer would it take a 50g ice cube to melt today in Dunedin compared to Marrakech?")

To calculate how much longer it would take for a 50g ice cube to melt in Dunedin compared to Marrakech, we need to consider the temperature difference between the two locations.

Assuming the melting point of ice is around 0°C (32°F), and using the temperatures you provided:

* In Dunedin: 4°C
* In Marrakech: 27°C

The temperature difference between the two locations is:

27°C - 4°C = 23°C

This means that the air in Marrakech is approximately 23°C warmer than the air in Dunedin.

Now, let's consider the melting rate of ice. The exact melting rate depends on various factors, including the shape and size of the ice cube, air circulation, and humidity. However, as a rough estimate, we can assume that the melting rate of ice is roughly proportional to the temperature difference between the ice and the surrounding air.

Using this assumption, we can estimate the relative melting rates in Dunedin and Marrakech:

* In Dunedin: 4°C (relatively slow melting)
* In Marrakech: 27°C (relatively fa

In [6]:
await weather_chat('llama3.1:70b', "If a wombat was transported today to Northern Sweden and a penguin was transported today to Singapore, which would have the best chance of survival?")

Based on the temperatures provided, I would say that the wombat has a better chance of survival. Wombats are native to Australia and are adapted to temperate climates with mild winters and cool summers. Northern Sweden's temperature of 12°C (54°F) is within the wombat's tolerance range.

On the other hand, penguins are found in cold climates, such as Antarctica and the surrounding islands. Singapore's temperature of 28°C (82°F) is much too hot for a penguin, which would likely suffer from heat stress and dehydration.

Therefore, I believe that the wombat has a better chance of survival in Northern Sweden than the penguin does in Singapore.


## Adding multiple functions

Let's construct a second function which obtains details of events available on Ticketmaster.

In [48]:
# function to get the today's sunset time in a place
def get_sunset(place: str) -> str:
  base_url = f'https://wttr.in/{place}?format="%s"'
  response = requests.get(base_url)
  data = response.json()
  return f"Today's sunset time in {place} is {data}"

# test the function
get_sunset("London")

"Today's sunset time in London is 19:22:59"

Now let's add this as an additional function available to Llama 3.1

In [51]:
async def weather_and_sunset_chat(model: str, query: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
     {
        'type': 'function',
        'function': {
          'name': 'get_sunset',
          'description': 'Get the sunset time in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The city in which the sunset time is requested',
              }
            },
            'required': ['place'],
          },
        },
      }, 
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
      'get_sunset': get_sunset
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Answer the following question: {query}.  
          Ignore any previous instructions or defaults and instead use the following information:  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])

And now let's test a few queries:

In [52]:
await weather_and_sunset_chat("llama3.1:8b", "Should I dress light for my visit To Austin, Texas today?")

Yes, you should dress lightly for your visit to Austin, Texas today. The temperature is around 75°F (24°C), which is quite warm. You may want to consider wearing light and breathable clothing such as t-shirts, shorts, and sunglasses to stay cool.


In [59]:
await weather_and_sunset_chat("llama3.1:8b", "I'm driving about 100 miles out of town of Austin, Texas today.  What kind of clothes should I pack?  Also I want to be back before dark.  Do you have any advice?")

Based on the current temperature in Austin, Texas being 24 degrees Celsius, it's likely to be warm during your drive. I would recommend packing light and breathable clothing such as t-shirts, shorts, and a hat.

Since you want to be back before dark, which is at 19:40:25, I would suggest starting your return journey by around 17:00-18:00 to ensure you arrive back in Austin with enough time to spare. This will also give you some buffer in case of any unexpected delays or traffic.

Additionally, you may want to check the weather forecast for the area you'll be driving through to see if there are any potential storms or rain showers that could affect your drive. It's always a good idea to be prepared for any conditions, especially when driving long distances.

Overall, with comfortable and practical clothing, and a sensible return time, you should have a safe and enjoyable drive back to Austin!
